In [23]:
from glob import glob
from os import path
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext
import time
import pickle

In [24]:
city = 'frank'

RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

In [25]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

#Subset to city
list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find(city) > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00']


In [26]:
base_file = 'frankfurt_db=0.50'
OC = 1.2
model = 'flex'
max_n_shifts = None

if max_n_shifts is None:
    results_file = f"../raw_results/{base_file}_OC={OC}_model={model}.pkl"
else:
    results_file = f"../raw_results/{base_file}_OC={OC}_model={model}_max_n_shifts={max_n_shifts}.pkl"

with open(results_file, 'rb') as file:
    sol = pickle.load(file)

In [28]:
# Costs
print('--- COSTS ---')
obj_val = sol['obj_val']
hiring_costs = sum( (employee, shift_start, day) in sol['r']
                   for region in sol['regions']
                   for employee in sol['employees'][region]
                   for day in sol['days']
                   for shift_start in sol['shifts'][(region, day)] )
outsourcing_costs = obj_val - hiring_costs
print(f'    Objective value: {obj_val} \n    Hiring costs: {hiring_costs} \n    Outsourcing costs: {outsourcing_costs}')

--- COSTS ---
    Objective value: 1498.8799999996988 
    Hiring costs: 288 
    Outsourcing costs: 1210.8799999996988


In [29]:
# Hours worked
print('--- HOURS WORKED ---')
people_hired = sum( 1
                   for region in sol['regions']
                   for employee in sol['employees'][region])

hours_worked_pct = sum(
    (employee, area, theta, day) in sol['k']
    for day in sol['days']
    for region in sol['regions']
    for employee in sol['employees'][region]
    for area in sol['reg_areas'][region]
    for theta in sol['periods'][day]
) / (people_hired * sol['hmax'] / 2) * 100 # hours per period 

print(f'    Overall hours worked pct: {hours_worked_pct}')

employee_hours = {}
for region in sol['regions']:
    for employee in sol['employees'][region]:
        employee_hours[employee] = sum(
            (employee, area, theta, day) in sol['k']
            for day in sol['days']
            for area in sol['reg_areas'][region]
            for theta in sol['periods'][day]
        ) / (sol['hmax'] / 2) * 100 # hours per period

print(f'    Mean hours pct per employee: {np.mean(list(employee_hours.values()))}')
print(f'    Std hours pct per employee: {np.std(list(employee_hours.values()))}')
print(f'    Quantile 25 hours pct per employee: {np.quantile(list(employee_hours.values()), 0.25)}')
print(f'    Median 25 hours pct per employee: {np.median(list(employee_hours.values()))}')
print(f'    Quantile 75 hours pct per employee: {np.quantile(list(employee_hours.values()), 0.75)}')

--- HOURS WORKED ---
    Overall hours worked pct: 68.57142857142857
    Mean hours pct per employee: 68.57142857142857
    Std hours pct per employee: 6.6325658462781565
    Quantile 25 hours pct per employee: 66.66666666666666
    Median 25 hours pct per employee: 66.66666666666666
    Quantile 75 hours pct per employee: 66.66666666666666


In [30]:
# Starting shifts 
print('--- SHIFTS STARTS ---')
employee_starts = []
# self.r[(employee, shift_start, day)]
for region in sol['regions']:
    for employee in sol['employees'][region]:
        for day in sol['days']:
            for shift_start in sol['shifts'][(region, day)]:
                if (employee, shift_start, day) in sol['r']:
                    employee_starts.append({'employee': employee, 'shift_start': shift_start, 'day': day})

employee_starts_df = pd.DataFrame(employee_starts)
emp_ = employee_starts_df.groupby("employee").agg({"shift_start": "nunique"})["shift_start"]
print(f'    Mean shifts start: {emp_.mean()}')
print(f'    Quantile 25 shifts start: {emp_.quantile(0.25)}')
print(f'    Quantile 50 shifts start: {emp_.quantile(0.5)}')
print(f'    Quantile 75 shifts start: {emp_.quantile(0.75)}')

print('--- SELECTED SHIFTS ---')
_ = employee_starts_df.groupby('shift_start').agg({'employee': 'nunique'})
num_employees = employee_starts_df['employee'].nunique()
print(f'    Utilization rate of shifts: {_["employee"].mean() / num_employees * 100}')

--- SHIFTS STARTS ---
    Mean shifts start: 2.342857142857143
    Quantile 25 shifts start: 2.0
    Quantile 50 shifts start: 2.0
    Quantile 75 shifts start: 3.0
--- SELECTED SHIFTS ---
    Utilization rate of shifts: 58.57142857142858


In [31]:
for base_file in list_city_dt_subset:
    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    # for OC in [1.2]:
        for model in ['fixed', 'flex', 'partflex']:
        # for model in ['fixed']:
            if model == 'partflex':
                for max_n_shifts in range(2,5):
                    # Save results
                    results_file = f"../raw_results/{base_file}_OC={OC}_model={model}_max_n_shifts={max_n_shifts}.pkl"
                    #with open(results_file, 'wb') as file:
                    #    pickle.dump(dict_results, file)
                    #print(f'instance_results saved to {results_file}')
            else:
                # Save results
                results_file = f"../raw_results/{base_file}_OC={OC}_model={model}.pkl"
                #with open(results_file, 'wb') as file:
                #    pickle.dump(dict_results, file)
                #print(f'instance_results saved to {results_file}')